# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя.

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [ ]:
%pip install razdel
%pip install pandas
%pip install scikit-learn

In [106]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import razdel
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.corpus import stopwords
import numpy as np
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
data = pd.read_csv('labeled.csv')

In [ ]:
def razdel_tokenize(text):
    tokens = razdel.tokenize(text)
    return [token.text for token in tokens]

In [24]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)
vectorizer_default = CountVectorizer()
X = vectorizer_default.fit_transform(train.comment)
X_test = vectorizer_default.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values

In [25]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X, y)

LogisticRegression(C=0.1, class_weight='balanced')

In [26]:
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.91      0.83      0.87       941
         1.0       0.73      0.85      0.79       501

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.83      1442
weighted avg       0.85      0.84      0.84      1442



In [28]:
probas = clf.predict_proba(X_test)
preds = (probas[:,1]>0.85).astype(int)
print(classification_report(y_test, preds, zero_division=0))


              precision    recall  f1-score   support

         0.0       0.72      0.99      0.83       941
         1.0       0.95      0.26      0.41       501

    accuracy                           0.74      1442
   macro avg       0.83      0.63      0.62      1442
weighted avg       0.80      0.74      0.68      1442



In [29]:
vectorizer_custom = CountVectorizer(tokenizer=razdel_tokenize)
X_custom = vectorizer_custom.fit_transform(train.comment)
X_custom_test = vectorizer_custom.transform(test.comment)

clf_custom = LogisticRegression(C=0.1, class_weight='balanced')
clf_custom.fit(X_custom, y)

preds = clf_custom.predict(X_custom_test)
print(classification_report(y_test, preds, zero_division=0))

probas = clf_custom.predict_proba(X_custom_test)
preds = (probas[:,1]>0.85).astype(int)
print(classification_report(y_test, preds, zero_division=0))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.90      0.82      0.86       941
         1.0       0.71      0.83      0.77       501

    accuracy                           0.83      1442
   macro avg       0.81      0.83      0.81      1442
weighted avg       0.84      0.83      0.83      1442

              precision    recall  f1-score   support

         0.0       0.72      0.99      0.84       941
         1.0       0.96      0.28      0.43       501

    accuracy                           0.75      1442
   macro avg       0.84      0.64      0.63      1442
weighted avg       0.80      0.75      0.70      1442



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Получается классификатор, обученный на векторизаторе с токенизацией от razdel'а, работает чуть-чуть лучше.

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [30]:
vectorizer_count = CountVectorizer(lowercase=True, tokenizer=razdel_tokenize, min_df=1, analyzer='word', ngram_range=(1,1))
vectorizer_tfidf = TfidfVectorizer(lowercase=True, tokenizer=razdel_tokenize, encoding='utf-8', use_idf=True, smooth_idf=True)

In [91]:
X = vectorizer_count.fit_transform(train.comment)
X_test = vectorizer_count.transform(test.comment)

clf_lr = LogisticRegression(C=0.1, class_weight='balanced')
clf_lr.fit(X, y)

probas = clf_lr.predict_proba(X_test)
preds = (probas[:,1]>0.85).astype(int)
print(classification_report(y_test, preds, zero_division=0))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.72      0.99      0.84       941
         1.0       0.96      0.28      0.43       501

    accuracy                           0.75      1442
   macro avg       0.84      0.64      0.63      1442
weighted avg       0.80      0.75      0.70      1442



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [45]:
X = vectorizer_tfidf.fit_transform(train.comment)
X_test = vectorizer_tfidf.transform(test.comment)

clf_mnb = MultinomialNB(alpha=0.1, fit_prior=False)
clf_mnb.fit(X, y)

probas = clf_mnb.predict_proba(X_test)
preds = (probas[:,1]>0.85).astype(int)
print(classification_report(y_test, preds, zero_division=0))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.77      0.99      0.87       941
         1.0       0.95      0.45      0.61       501

    accuracy                           0.80      1442
   macro avg       0.86      0.72      0.74      1442
weighted avg       0.83      0.80      0.78      1442



## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов.

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [147]:
list_of_stopwords = list(stopwords.words('russian'))
list_of_stopwords += list('?!.,();:...-')

In [105]:
vectorizer_count = CountVectorizer(stop_words=list_of_stopwords, tokenizer=razdel_tokenize)
X = vectorizer_count.fit_transform(train.comment)
X_test = vectorizer_count.transform(test.comment)

clf_lr = LogisticRegression(C=0.1, class_weight='balanced')
clf_lr.fit(X, y)
clf_lr.classes_

preds = clf_lr.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

toxic5 = []

for i in np.sort(clf_lr.coef_[0])[::-1][:5]:
  toxic5.append(list(clf_lr.coef_[0]).index(i))
vectorizer_count.get_feature_names_out()[toxic5]

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

         0.0       0.89      0.82      0.85       941
         1.0       0.70      0.81      0.76       501

    accuracy                           0.82      1442
   macro avg       0.80      0.82      0.80      1442
weighted avg       0.83      0.82      0.82      1442



array(['хохлы', 'хохлов', 'тебе', 'пиздец', 'нахуй'], dtype=object)

In [136]:
vectorizer_count = CountVectorizer(stop_words=list_of_stopwords, tokenizer=razdel_tokenize)
X = vectorizer_count.fit_transform(train.comment)
X_test = vectorizer_count.transform(test.comment)

clf_dtree = DecisionTreeClassifier(max_depth=500, class_weight='balanced')
clf_dtree.fit(X, y)
clf_dtree.classes_

preds = clf_dtree.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

toxic5 = []

for i in np.sort(clf_dtree.feature_importances_)[::-1][:5]:
  toxic5.append(list(clf_dtree.feature_importances_).index(i))
vectorizer_count.get_feature_names_out()[toxic5]

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.84      0.76      0.80       941
         1.0       0.62      0.72      0.66       501

    accuracy                           0.75      1442
   macro avg       0.73      0.74      0.73      1442
weighted avg       0.76      0.75      0.75      1442



array(['очень', 'тебе', 'хохлы', 'хохлов', 'нахуй'], dtype=object)

In [176]:
vectorizer_count = CountVectorizer(stop_words=list_of_stopwords, tokenizer=razdel_tokenize)
X = vectorizer_count.fit_transform(train.comment)
X_test = vectorizer_count.transform(test.comment)

clf_mnb = MultinomialNB(alpha=0.1)
clf_mnb.fit(X, y)
clf_mnb.classes_

preds = clf_mnb.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

toxic5 = []

for i in np.sort(clf_mnb.feature_log_prob_[1])[::-1][:5]:
  toxic5.append(list(clf_mnb.feature_log_prob_[1]).index(i))
vectorizer_count.get_feature_names_out()[toxic5]

# странная выдача, честно говоря, но у меня не получилось найти аналог .coef_ для этого и других байесовских классификаторов

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.88      0.93      0.90       941
         1.0       0.84      0.76      0.80       501

    accuracy                           0.87      1442
   macro avg       0.86      0.84      0.85      1442
weighted avg       0.87      0.87      0.87      1442



array(['это', 'просто', '...', 'тебе', 'всё'], dtype=object)

In [139]:
vectorizer_count = CountVectorizer(stop_words=list_of_stopwords, tokenizer=razdel_tokenize)
X = vectorizer_count.fit_transform(train.comment)
X_test = vectorizer_count.transform(test.comment)

clf_randfor = RandomForestClassifier(n_estimators=100, max_depth=100, )
clf_randfor.fit(X, y)
clf_randfor.classes_

preds = clf_randfor.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

toxic5 = []

for i in np.sort(clf_randfor.feature_importances_)[::-1][:5]:
  toxic5.append(list(clf_randfor.feature_importances_).index(i))
vectorizer_count.get_feature_names_out()[toxic5]

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.73      0.99      0.84       941
         1.0       0.95      0.30      0.45       501

    accuracy                           0.75      1442
   macro avg       0.84      0.64      0.64      1442
weighted avg       0.81      0.75      0.70      1442



array(['хохлов', 'хохлы', 'тебе', 'нахуй', 'очень'], dtype=object)